# Introduction

This Jupyter Notebook demonstrates the step-by-step process for loading and utilizing an existing MODFLOW model, specifically the **COHYST** (Cooperative Hydrology Study) model. The COHYST model is widely used to simulate groundwater flow and assess water resource management strategies in Nebraska. 

The notebook leverages the capabilities of the **Flopy** library, a Python package for working with MODFLOW models. It includes tasks such as:

- Loading the model's input and output files,
- Running the model,
- Visualizing hydraulic heads and other key results,
- Extracting meaningful insights from the data.

By following this notebook, users can efficiently analyze an existing MODFLOW model and adapt similar workflows to their own groundwater modeling projects.
 projects.


# Resources

- **[Flopy Documentation](https://flopy.readthedocs.io/en/latest/introduction.html)**
- **[MODFLOW Online Documentation](https://water.usgs.gov/nrp/gwsoftware/modflow2000/MFDOC/index.html?rch.htm)**
- **[MODFLOW-2005 User Manual](https://water.usgs.gov/ogw/modflow/MODFLOW-2005-Guide/index.html)**
- **[Cooperative Hydrology Study (COHYST)](https://cohyst.nebraska.gov/)**

---

**Note:**  
First, create your working folder and label it clearly 

Next, find and download the COHYST Groundwater model from the **COHYST website.(https://cohyst.nebraska.gov/#mds).** 

Check the model files within the folder labeled 'MODFLOW' that you downloaded from the COHYST website. You will discover that the different files are labeled 'cohyst_template' followed by the extension (file type).

The quotation marks around 'cohyst_template' indicate that the name is a placeholder. This means it represents a generic or temporary name that you can customize to suit your specific project.

You have two options when working with this placeholder:

1. Keep the name: If you decide to use the name as it is, simply remove the quotation marks (') so that the name becomes cohyst_template without the quotes.
2. Change the name: If you want to replace the placeholder with your own name, ensure you also remove the quotation marks (') and use your desired name without them.


Additionally, to use this script, you must download a MODFLOW executable from the [USGS website](https://water.usgs.gov/ogw/modflow/) and place it in the same folder as the model. Remember that this script will only work with MODFLOW or MODFLOW-2005 executables and may be modified to use MODFLOW-USG. **MODFLOW 6** has a separate set of Flopy


### I. Import Packages and Create Model Object

In [ ]:
import flopy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
import os

%matplotlib inline

In [ ]:
#Check Flopy version to ensure it is the latest
flopy.__version__


In [ ]:
#Double-check versions for others too if needed
import sys
print(sys.version)
print(f"numpy version: {np.__version__}")
print(f"matplotlib version: {mp.__version__}")
print(f"flopy version: {flopy.__version__}")

In [ ]:
#Set workspace
workspace = "path_to_your_data" 
if not os.path.exists(workspace):
    os.makedirs(workspace)

print(f"Workspace directory set to: {workspace}")

In [ ]:
files = os.listdir(workspace)


print(f"Files in the directory '{workspace}':") # Print the list of files
for file in files:
    file_path = os.path.join(workspace, file)
    if os.path.isfile(file_path):
        print(f"File: {file}")
    elif os.path.isdir(file_path):
        print(f"Directory: {file}")    
        
      
        
only_files = [f for f in files if os.path.isfile(os.path.join(workspace, f))]
print("\nList of files only:")
for f in only_files:
    print(f)


if 'cohyst_template.nam' in files:
    print(' cohyst_template is present')
else:
    print('It is not there')

In [ ]:
# Define the workspace and model name if you haven't already done so
model_ws=workspace
modelname = "cohyst_template"


In [ ]:
#Checking that files are indeed present in the workspace. You can test out the various files from the previous step

dis_file = os.path.join(model_ws, 'cohyst_template.dis')  # replace with your actual DIS file name
rch_file = os.path.join(model_ws, 'cohyst_template.rch')  # replace with your actual RCH file name
hds_file = os.path.join(model_ws, 'cohyst_template.hds')  # replace with your actual HDS file name
lst_file = os.path.join(model_ws, 'cohyst_template.lst')  # replace with your actual LST file name
cbb_file = os.path.join(model_ws, 'cohyst_template.cbb')  # replace with your actual CBB file name
ddn_file = os.path.join(model_ws, 'cohyst_template.ddn')  # replace with your actual DDN file name
obh_file = os.path.join(model_ws, 'cohyst_template_obh.out') # replace with your actual OBH OUT file name
sfr_file = os.path.join(model_ws, 'cohyst_template_sfr.out') # replace with your actual SFR OUT file name

# Check if files exist
print(os.path.isfile(dis_file))  # Should print True if the file exists
print(os.path.isfile(rch_file))  # Should print True if the file exists
print(os.path.isfile(hds_file))  # Should print True if the file exists
print(os.path.isfile(lst_file))  # Should print True if the file exists
print(os.path.isfile(cbb_file))  # Should print True if the file exists
print(os.path.isfile(ddn_file))  # Should print True if the file exists
print(os.path.isfile(obh_file))  # Should print True if the file exists
print(os.path.isfile(sfr_file))  # Should print True if the file exists


After running the above codes, it appears that some files are missing from the COHYST MODFLOW folder (.lst, obh_out, sfr_out,.cbb, .hds, .ddn)
This happened at the initial stages because they are all output files which are typically created after a MODFLOW model run. Each file is generated as part of the output process when the model simulation is executed successfully.

LST - listing file - provides log of the entire sim including input data and model performance

OBH_OUT - observed head output - shows how the model's heads compare with the observed values

SFR_OUT - streamflow routing output - shows streamflow results + interactions between surface and groundwater

CBB - cell-by-cell budget file - includes detailed breakdown of the water budget for each cell tracking inflows and outflows to give understanding of          flow processes within the model cell

DDN - drawdown file - shows the computed drawdown for each model cell allowing the analysis of effects of groundwater pumping/other stress on the             aquifer  

HDS - heads file - contains the simulated hydraulic heads for each cell in the model domain which helps in analyzing groundwater flow patterns


In [ ]:
#This step may not be necessary after model has been successfully run as it will be in the upcoming steps
#Load the model NAM file first before trying to explore the other files
ml = flopy.modflow.Modflow.load(
    "cohyst_template.nam",
    model_ws=workspace,
    verbose=True,
    version="mf2005",
    check=False,
)

Next we will try to load the files one by one to explore them and determine if there are any additional steps required

In [ ]:
# Load the DIS file - this cell is for reference only - running it results in an error

dis = flopy.modflow.ModflowDis.load(dis_file, ml)
model_name = "cohyst_template"
model_ws=workspace
# Explore the contents
print(dis)
print(dis.nrow, dis.ncol, dis.nlay)  # print the number of rows, columns, and layers

DIS load error that results from running the above cell is addressed later in the notebook. At this stage it results in an error likely caused by an issue with the ext_unit_dict dictionary or the file handle that flopy expects to use for loading external files.

In [ ]:
# Load the RCH file
rch = flopy.modflow.ModflowRch.load(rch_file, ml)
if rch is None:
    raise ValueError("Failed to load the RCH file.")
else:
    print(rch)
    if rch.rech is None or rch.rech.shape[0] == 0:
        raise ValueError("The recharge array is empty or not correctly loaded.")
    else:
        print(f"Recharge array dimensions: {rch.rech.shape}")
        print(rch.rech)  # Print the recharge array

The next steps help address the error that resulted when trying to load the DIS FIle above. We have to check the formats of some files to determine how best to access them

In [ ]:
# The line below prints out the format of the hydraulic conductivity array for the first layer.
print(ml.lpf.hk[0].format)

# Solution
Resetting the format using a standard Fortran-type format descriptor in MODFLOW is necessary for controlling how input and output data are read and written by the program. MODFLOW, originally written in Fortran, relies on these format descriptors to interpret and process data in the correct format, particularly when dealing with formatted text files for inputs or outputs.
Source: https://flopy.readthedocs.io/en/latest/Notebooks/array_output_tutorial.html 

In [ ]:
print(ml.dis.botm[0].format.fortran) #This prints the Fortran format string for the array, which is how the data would be formatted if written in Fortran, often used by MODFLOW.
print(ml.dis.botm[0].format.py) #This prints the Python format string for the array, showing how the data is represented in Python.
print(ml.dis.botm[0].format.numpy) #This prints the NumPy format string, which indicates how the data is represented in NumPy arrays, a core library in Python for numerical computations.

In [ ]:
# Reset the format using a standard fortran type format descriptor
ml.dis.botm[0].format.free = False
ml.dis.botm[0].format.fortran = "(20f10.4)"
print(ml.dis.botm[0].format.fortran)
print(ml.dis.botm[0].format.py)
print(ml.dis.botm[0].format.numpy)
print(ml.dis.botm[0].format)

Try the DIS file load again after the above steps

In [ ]:
# Load the DIS file 
dis = flopy.modflow.ModflowDis.load(dis_file, ml)
model_name = "cohyst_template"
model_ws=workspace
# Explore the contents
print(dis)
print(dis.nrow, dis.ncol, dis.nlay)  # print the number of rows, columns, and layers

**Now the model is ready and can be run**

In [ ]:
#Write input and run model
ml.write_input()
success, buff = ml.run_model(silent=True, report=True)
if success:
    for line in buff:
        print(line)
else:
    raise ValueError("Failed to run.")

NB: Once the model has been run, we can now explore it further in the next steps

In [ ]:

# Let's try to plot the grid
fig, ax = plt.subplots(figsize=(10, 10))
mf.modelgrid.plot(ax=ax)

# Display the plot
plt.title('MODFLOW Model Grid')
plt.show()

**What happened?**

The plot appears as a solid grey rectangle because the grid representation in the MODFLOW model might not be fully configured or is not displaying cell boundaries properly. This can happen if:

Inactive Cells Are Not Displayed: If the plot only shows active cells, and the grid is entirely inactive or undifferentiated, it may appear as a solid color.

Grid Data Not Loaded Properly: If the MODFLOW model does not contain active grid data or if the grid boundaries are not defined, it will show up as a solid block.

Plot Parameters: If specific plot parameters such as plot_bc() or plot_array() are not used, the plot may only display the outline of the model grid instead of individual cells or attributes.

In [ ]:
#You can first try to check for active and inactive cells
mf.dis.idomain.array  # Check the array of active/inactive cells

**You will likely get an error when you run the above cell**

The AttributeError indicates that the ModflowDis object does not have an idomain attribute, which typically stores information about active and inactive cells. 

This is common in older versions of MODFLOW (this is 2005) or FloPy (flopy is up-to-date) where the idomain attribute might not be directly accessible.

**What steps can we take?**

Verify Model Version and Array Name: Depending on the version of MODFLOW and FloPy, the attribute names for active cells may differ.

In [ ]:
#print available attributes to confirm:
print(dir(mf.dis))  # List available attributes in the discretization package


If you encounter an AttributeError for the ibound attribute, it could be because the ibound array is not explicitly named as such within the model or is accessed differently in your setup. In MODFLOW 2005, ibound is usually included with the discretization package but is sometimes referred to in FloPy with other specific attributes or may need to be constructed manually.

Below is how to work around this:

- Access the bas6 Package for IBOUND:
The ibound array is often stored in the basic package (bas6) of MODFLOW 2005. 

In [ ]:
# Check for Basic Package Existence and if bas6 is listed then use the next cell code to access
print(mf.get_package_list())  # This will list all packages associated with your model

In [ ]:
# Now access ibound array in modflow2005 is 

ibound = mf.bas6.ibound.array  # Access the ibound array from the basic package


**Now you can plot the active and inactive cells!** 

In [ ]:
import matplotlib.pyplot as plt
import flopy

fig, ax = plt.subplots(figsize=(10, 10))
map_view = flopy.plot.PlotMapView(model=mf, ax=ax)
map_view.plot_array(ibound[0])  # Plot the first layer; adjust the index as needed for other layers
plt.title('Active Cells in MODFLOW Model Grid - Layer 1')
plt.show()

**You can plot the hydraulic conductivity**

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
conductivity = mf.lpf.hk.array  # Hydraulic conductivity in MODFLOW 2005
map_view = flopy.plot.PlotMapView(model=mf, ax=ax)
map_view.plot_array(conductivity[0])  # Layer 1, adjust the index as needed
plt.title('Hydraulic Conductivity in MODFLOW Model Grid - Layer 1')
plt.show()

In [ ]:
# Get well data from the WEL package
if 'WEL' in mf.get_package_list():
    wel = mf.wel
    wel_data = wel.stress_period_data[0]  # Get the well data for the first stress period
    well_locations = [(rec['i'], rec['j']) for rec in wel_data]  # Extract the row, col locations

# Plot the model grid
fig, ax = plt.subplots(figsize=(10, 10))
mf.modelgrid.plot(ax=ax)

# Plot well locations on the grid
for loc in well_locations:
    ax.plot(loc[1], loc[0], 'ro', label='Well')  # loc[1] is column (x), loc[0] is row (y)

plt.title('MODFLOW Model Grid with Wells')
plt.show()

**The above steps may result in a gray plot too so we have some additional steps to fix this**

In [ ]:
# Check if the Well (WEL) package exists in the model
package_list = mf.get_package_list()
if 'WEL' in package_list:
    print("WEL package exists in the model.")
    
    # Access Well package data
    wel = mf.get_package('WEL')
    wel_data = wel.stress_period_data  # Get well data for all stress periods
    print(f"Well data for first stress period: {wel_data[0]}")
else:
    print("WEL package does not exist in the model.")

In [ ]:
# ---- Extract Head Data ----
# Load the head file (.hds file)
hds = flopy.utils.HeadFile(f'{model_ws}/{model_name}.hds')

# Check available time steps for head data
times = hds.get_times()
print(f"Available times in head file: {times}")

# Get head data for the final time step (or specify a time step)
if len(times) > 0:
    head_data = hds.get_data(totim=times[-1])  # Extract head data for the final time step
    print("Head data for the final time step:")
    print(head_data)
else:
    print("No times available in head file.")

# ---- Extract Flow Data ----
# Load the cell-by-cell budget file (.cbb file)
cbb = flopy.utils.CellBudgetFile(f'{model_ws}/{model_name}.cbb')

# Check available time steps for flow data
cbb_times = cbb.get_times()
print(f"Available times in cell-by-cell budget file: {cbb_times}")

# List available record types (e.g., 'FLOW RIGHT FACE', 'RECHARGE')
record_names = cbb.get_unique_record_names()
print(f"Available record names in .cbb file: {record_names}")

# Check if there are valid times and flow records
if len(cbb_times) > 0 and 'FLOW RIGHT FACE' in record_names:
    # Get flow data for the final time step
    flow_data = cbb.get_data(text='FLOW RIGHT FACE', totim=cbb_times[-1])  # Final time step
    print("Flow data for 'FLOW RIGHT FACE' for the final time step:")
    print(flow_data)
else:
    print("No valid flow data available for 'FLOW RIGHT FACE' or no time steps available.")

In [ ]:
# Plot head data for the first layer (layer=0)
layer = 0
head_layer = head_data[layer, :, :]  # Extract data for the specified layer

# Create a plot of the head data
fig, ax = plt.subplots(figsize=(10, 10))
c = ax.contourf(head_layer, cmap='viridis')  # Use contour plot to visualize head
plt.colorbar(c, ax=ax, label='Head (m)')
plt.title(f'Head Distribution for Layer {layer} (Final Time Step)')
plt.xlabel('Column')
plt.ylabel('Row')
#plt.show()
plt.savefig('C:/Users/Brend/Downloads/head_distribution_layer.png', format='png', dpi=300)

In [ ]:
# Assuming well data exists in the 'WEL' package
if 'WEL' in mf.get_package_list():
    wel = mf.get_package('WEL')
    wel_data = wel.stress_period_data[0]  # Get well data for the first stress period
    well_locations = [(rec['i'], rec['j']) for rec in wel_data]  # Extract row, col locations

    # Plot well locations on top of the head data
    fig, ax = plt.subplots(figsize=(10, 10))
    c = ax.contourf(head_layer, cmap='viridis')
    plt.colorbar(c, ax=ax, label='Head (m)')

    # Plot wells as red dots
    for loc in well_locations:
        row, col = loc
        ax.plot(col, row, 'ro', markersize=8, label='Well')

    plt.title(f'Head Distribution with Wells for Layer {layer}')
    plt.xlabel('Column')
    plt.ylabel('Row')
    #plt.show()
    plt.savefig('C:/Users/Brend/Downloads/head_distribution_with_wells_layer.png', format='png', dpi=300)

In [ ]:
# Checking information on stress periods in COHYST model
# Load the discretization package
dis = mf.get_package('DIS')

# Check the number of stress periods
nper = dis.nper
print(f"Number of stress periods: {nper}")

# Print the details of each stress period
print("Stress Period Details:")
for i in range(nper):
    print(f"  Stress Period {i + 1}:")
    print(f"    Length: {dis.perlen[i]}")
    print(f"    Time Steps: {dis.nstp[i]}")
    print(f"    Steady-State: {dis.steady[i]}")


In [ ]:
# To check the time units first Load the discretization package
dis = mf.get_package('DIS')

# Check the time units
time_units_code = dis.itmuni
time_units = {0: "undefined", 1: "seconds", 2: "minutes", 3: "hours", 4: "days", 5: "years"}

# Display the time unit
print(f"Time units: {time_units.get(time_units_code, 'unknown')}")


In [ ]:
# Access the GHB - General-Head Boundary package - this will give idea of how boundary conditions are influencing GW flow
ghb = mf.get_package('GHB')

# Display the stress period data (head, conductance, and cell locations)
ghb_data = ghb.stress_period_data  # Dictionary containing boundary data for each stress period

# For example, to view data for the first stress period
print("GHB Data for Stress Period 1:")
print(ghb_data[0])
# The output  is (layer, row, column, head (units?), conductance)

In [ ]:
#Exploring well package
# Access the WEL package
wel = mf.get_package('WEL')

# Get the stress period data
well_data = wel.stress_period_data  # Dictionary with data for each stress period

# Example: Print well data for the first stress period (0)

print("Well Data for Stress Period 1:")
print(well_data[0])

#should show list of wells (layer, row, column, pumping rate + is injection and - is extraction)

In [ ]:
# Assuming 'RECHARGE' is one of the boundary conditions in the cbb file
if 'RECHARGE' in cbb.get_unique_record_names():
    # Get recharge data for the final time step (if available - essential for me to find this out for cohyst)
    recharge_data = cbb.get_data(text='RECHARGE')
    if recharge_data:
        recharge_array = recharge_data[0]  # Extract the first layer of recharge data

        # Plot recharge on top of the head data
        fig, ax = plt.subplots(figsize=(10, 10))
        c = ax.contourf(head_layer, cmap='viridis')
        plt.colorbar(c, ax=ax, label='Head (m)')

        # Plot recharge as a contour plot
        r = ax.contour(recharge_array, colors='blue', levels=5)
        plt.colorbar(r, ax=ax, label='Recharge (m/day)')
        
        plt.title(f'Head Distribution with Recharge for Layer {layer}')
        plt.xlabel('Column')
        plt.ylabel('Row')
        #plt.show()
        plt.savefig('C:/Users/Brend/Downloads/head_distribution_with_recharge_layer.png', format='png', dpi=300)
    else:
        print("No recharge data available.")

<strong> You are all set! </strong> You can continue to explore the model as you determine how best to use it to suit your objectives

